In [1]:
%load_ext autoreload
%autoreload 2 

In [2]:
import numpy as np 
import pandas as pd

In [3]:
dataset_path = "./dataset.csv"

df = pd.read_csv(
    dataset_path,
    # dtype is specified to prevent a DtypeWarning
    dtype={
        "full_query": str,
        "label": int,
        "statement_type": str,
        "query_template_id": str,
        "attack_payload": str,
        "attack_id": str,
        "attack_technique": str,
        "attack_desc": str,
        "split": str,
        "sqlmap_status" : str,
        "attack_stage" : str,
    },
)


In [4]:
def display_counts_recon(df : pd.DataFrame):
    result = df.groupby('attack_id')['attack_stage'].value_counts().unstack(fill_value=0)
    with pd.option_context('display.max_rows', None, 
                       'display.max_columns', None,
                       'display.width', None,
                       'display.max_colwidth', None):
        display(result)
_ = display_counts_recon(df)


attack_stage,exploit,recon
attack_id,,
error-0,0,10
error-1,0,4
error-2,0,15
error-3,0,9
error-4,0,3
error-5,0,25
error-6,0,10
error-7,2,40
error-8,0,8


In [ ]:
_df = df.loc[(df["attack_id"] == "error-7")]
with pd.option_context('display.max_rows', None, 
                    'display.max_columns', None,
                    'display.width', None,
                    'display.max_colwidth', None):
    display(_df)

,full_query,label,attack_payload,attack_stage,tamper_method,attack_status,attack_desc,statement_type,query_template_id,attack_id,attack_technique,split
10,"SELECT a.id, a.ident, a.name, a.latitude_deg, a.longitude_deg,(6371 * ACOS(COS(RADIANS(a.latitude_deg)) * COS(RADIANS(b.latitude_deg)) * COS(RADIANS(b.longitude_deg) - RADIANS(a.longitude_deg)) + SIN(RADIANS(a.latitude_deg)) * SIN(RADIANS(b.latitude_deg)))) AS distance FROM airport a JOIN airport b ON b.name = ""(SELECT\rCONCAT(CONCAT(0x7176787171,(CASE WHEN (4956=4956)\tTHEN 0x31\tELSE\n0x30 END)),0x717a7a7a71))"" WHERE a.id != b.id ORDER BY distance LIMIT 50;",1,NaN,recon,space2mysqlblank,failure,Recognition payload using technique error,select,airport-S14,error-4,error,test
32,"SELECT a.id, a.ident, a.name, a.latitude_deg, a.longitude_deg,(6371 * ACOS(COS(RADIANS(a.latitude_deg)) * COS(RADIANS(b.latitude_deg)) * COS(RADIANS(b.longitude_deg) - RADIANS(a.longitude_deg)) + SIN(RADIANS(a.latitude_deg)) * SIN(RADIANS(b.latitude_deg)))) AS distance FROM airport a JOIN airport b ON b.name = ""(SELECT\nCONCAT(0x7176787171,(ELT(8995=8995,1)),0x717a7a7a71))"" WHERE a.id != b.id ORDER BY distance LIMIT 50;",1,NaN,recon,space2mysqlblank,failure,Recognition payload using technique error,select,airport-S14,error-4,error,test
58,"SELECT a.id, a.ident, a.name, a.latitude_deg, a.longitude_deg,(6371 * ACOS(COS(RADIANS(a.latitude_deg)) * COS(RADIANS(b.latitude_deg)) * COS(RADIANS(b.longitude_deg) - RADIANS(a.longitude_deg)) + SIN(RADIANS(a.latitude_deg)) * SIN(RADIANS(b.latitude_deg)))) AS distance FROM airport a JOIN airport b ON b.name = ""McDonald\tHeliport'ywaJll<'"">rCFIDM"" WHERE a.id != b.id ORDER BY distance LIMIT 50;",1,NaN,recon,space2mysqlblank,failure,Recognition payload using technique error,select,airport-S14,error-4,error,test


In [ ]:
# ASR = Attack success rate
def display_asr_per_technique(df : pd.DataFrame):
    df_a = df[df["label"] == 1]
    # First keep a sample for each ID: 
    dfs = df_a.groupby('attack_id').sample(n=1, random_state=42)
    # Then, compute for each technique, the success 
    r = dfs["sqlmap_status"].value_counts(dropna=False)
    print(r)

# display_asr_per_technique(df=df)

KeyError: 'sqlmap_status'

In [ ]:
def display_NAN_sqlmap_status(df : pd.DataFrame): 
    df_a = df[df["label"] == 1]
    dfa_nan = df_a[df_a["sqlmap_status"].isna()]
    display(dfa_nan)

def fix_NAN_sqlmap_status(df : pd.DataFrame):
    dfa = df[df["label"] == 1]
    for attack in dfa["attack_id"].unique():
        dfa_aid = dfa[dfa["attack_id"] == attack]
        status = dfa_aid.loc[dfa_aid["attack_stage"] == "exploit"].iloc[0]["sqlmap_status"]
        print(status)
# fix_NAN_sqlmap_status(df=df)

success
success
success
success
success
success
success
success
success
success
success
success
success
failure
success
success
failure
success
failure
success
success
failure
failure
success
success
failure
failure
success
success
failure
success
success
success
success
failure
success
success
success
success
success
success
success
success
failure
success
success
success
success
failure
success
success
success
success
success
success
failure
success
failure
failure
success
success
success
failure
success
success
success
failure
success
failure
success
success
failure
success
success
success
failure
success
failure
failure
failure
success
success
success
success
success
success
success
failure
failure
failure
success
success
success
failure
success
success
success
failure
success
failure
success
success
success
failure
success
success
failure
success
failure
success
success
failure
failure
success
success
success
success
success
failure
success
success
success
success
success
success


IndexError: single positional indexer is out-of-bounds